In [2]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT, force_remount=True)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [3]:
!pip install bert-for-tf2

     |████████████████████████████████| 40kB 2.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.6-cp36-none-any.whl size=30318 sha256=6af0f286184f7676e531f57a1e3e58a37f62cd325f3fae9eacef3741cc883630
  Stored in directory: /root/.cache/pip/wheels/07/a0/b4/75b0601ebaa41e517a797fe9cea119c789664c8408f8a74ae9
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=9ac2f6ac8dd9e8e32297e10963c98c2a1824547dec2fc2eb075cd45cd7cfdad4
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=9a824839911b9b5866c5dbac218a2701c198a389c19b13fb45ebc863c9fd429d
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [4]:
%pwd
%cd 'drive/My Drive/Workspaces/twitter_sentiment_analysis'

/content/drive/My Drive/Workspaces/twitter_sentiment_analysis


In [5]:
import nltk
import re

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 

import gc
import re
import string
import operator

import matplotlib.pyplot as plt
import seaborn as sns

# import tokenization
from wordcloud import STOPWORDS

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import bert

nltk.download('wordnet')
nltk.download('stopwords')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('data/preprocessed_train.csv')
test_df = pd.read_csv('data/preprocessed_test.csv')

In [16]:
HateSpeechTweets = train_df['label'] == 1

SEED = 1337
K = 2
skf = StratifiedKFold(n_splits=K, random_state=SEED, shuffle=True)

print('Whole Training Set Shape = {}'.format(train_df.shape))
print('Whole Training Set Target Rate (Hate Speech) {}/{} (Not Hate Speech)'.format(train_df[HateSpeechTweets]['label'].count(), train_df[~HateSpeechTweets]['label'].count()))

for fold, (trn_idx, val_idx) in enumerate(skf.split(train_df['cleaned_text'], train_df['label']), 1):
    print('\nFold {} Training Set Shape = {} - Validation Set Shape = {}'.format(fold, train_df.loc[trn_idx, 'cleaned_text'].shape, train_df.loc[val_idx, 'cleaned_text'].shape))

Whole Training Set Shape = (31962, 14)
Whole Training Set Target Rate (Hate Speech) 2242/29720 (Not Hate Speech)

Fold 1 Training Set Shape = (15981,) - Validation Set Shape = (15981,)

Fold 2 Training Set Shape = (15981,) - Validation Set Shape = (15981,)


In [8]:
class ClassificationReport(Callback):
  def __init__(self, train_data=(), validation_data=()):
    super(Callback, self).__init__()

    self.X_train, self.y_train = train_data
    self.train_precision_scores = []
    self.train_recall_scores = []
    self.train_f1_scores = []

    self.X_val, self.y_val = validation_data
    self.val_precision_scores = []
    self.val_recall_scores = []
    self.val_f1_scores = [] 
               
  def on_epoch_end(self, epoch, logs={}):    
    train_predictions = np.round(self.model.predict(self.X_train, verbose=0))        
    train_precision = precision_score(self.y_train, train_predictions, average='macro')
    train_recall = recall_score(self.y_train, train_predictions, average='macro')
    train_f1 = f1_score(self.y_train, train_predictions, average='macro')
    self.train_precision_scores.append(train_precision)        
    self.train_recall_scores.append(train_recall)
    self.train_f1_scores.append(train_f1)

    val_predictions = np.round(self.model.predict(self.X_val, verbose=0))
    val_precision = precision_score(self.y_val, val_predictions, average='macro')
    val_recall = recall_score(self.y_val, val_predictions, average='macro')
    val_f1 = f1_score(self.y_val, val_predictions, average='macro')
    self.val_precision_scores.append(val_precision)        
    self.val_recall_scores.append(val_recall)        
    self.val_f1_scores.append(val_f1)

    print('\nEpoch: {} - Training Precision: {:.6} - Training Recall: {:.6} - Training F1: {:.6}'.format(epoch + 1, train_precision, train_recall, train_f1))
    print('Epoch: {} - Validation Precision: {:.6} - Validation Recall: {:.6} - Validation F1: {:.6}'.format(epoch + 1, val_precision, val_recall, val_f1))  

In [9]:
%%time
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1', trainable=True)

CPU times: user 9.73 s, sys: 1.7 s, total: 11.4 s
Wall time: 16.5 s


In [ ]:
train_df['cleaned_text'] = train_df.cleaned_text.apply(lambda text : text.strip())
test_df['cleaned_text'] = test_df.cleaned_text.apply(lambda text : text.strip())


In [38]:
class DisasterDetector:
  def __init__(self, bert_layer, max_seq_length=128, lr=0.0001, epochs=15, batch_size=32):        
    # BERT and Tokenization params
    self.bert_layer = bert_layer

    self.max_seq_length = max_seq_length        
    vocab_file = self.bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = self.bert_layer.resolved_object.do_lower_case.numpy()
    self.tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

    # Learning control params
    self.lr = lr
    self.epochs = epochs
    self.batch_size = batch_size

    self.models = []
    self.scores = {}

  def encode(self, texts):

    all_tokens = []
    all_masks = []
    all_segments = []

    for text in texts:
      print(text)
      text = self.tokenizer.tokenize(text)
      text = text[:self.max_seq_length - 2]
      input_sequence = ['[CLS]'] + text + ['[SEP]']
      pad_len = self.max_seq_length - len(input_sequence)

      tokens = self.tokenizer.convert_tokens_to_ids(input_sequence)
      tokens += [0] * pad_len
      pad_masks = [1] * len(input_sequence) + [0] * pad_len
      segment_ids = [0] * self.max_seq_length

      all_tokens.append(tokens)
      all_masks.append(pad_masks)
      all_segments.append(segment_ids)

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)


  def build_model(self):
    input_word_ids = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='input_word_ids')
    input_mask = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='input_mask')
    segment_ids = Input(shape=(self.max_seq_length,), dtype=tf.int32, name='segment_ids')    

    pooled_output, sequence_output = self.bert_layer([input_word_ids, input_mask, segment_ids])   
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    optimizer = SGD(learning_rate=self.lr, momentum=0.8)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model


  def train(self, X):

    for fold, (trn_idx, val_idx) in enumerate(skf.split(X['cleaned_text'], X['hashTags'])):
      print('\nFold {}\n'.format(fold))
      print(trn_idx)
      X_trn_encoded = self.encode(X.loc[trn_idx, 'cleaned_text'].str.lower())
      y_trn = X.loc[trn_idx, 'label']
      X_val_encoded = self.encode(X.loc[val_idx, 'cleaned_text'].str.lower())
      y_val = X.loc[val_idx, 'label']

      # Callbacks
      metrics = ClassificationReport(train_data=(X_trn_encoded, y_trn), validation_data=(X_val_encoded, y_val))

      # Model
      model = self.build_model()        
      model.fit(X_trn_encoded, y_trn, validation_data=(X_val_encoded, y_val), callbacks=[metrics], epochs=self.epochs, batch_size=self.batch_size)

      self.models.append(model)
      self.scores[fold] = {
          'train': {
              'precision': metrics.train_precision_scores,
              'recall': metrics.train_recall_scores,
              'f1': metrics.train_f1_scores                    
          },
          'validation': {
              'precision': metrics.val_precision_scores,
              'recall': metrics.val_recall_scores,
              'f1': metrics.val_f1_scores                    
          }
      }


  def plot_learning_curve(self):
    fig, axes = plt.subplots(nrows=K, ncols=2, figsize=(20, K * 6), dpi=100)

    for i in range(K):
      # Classification Report curve
      sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[i].history.history['val_accuracy'], ax=axes[i][0], label='val_accuracy')
      sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['precision'], ax=axes[i][0], label='val_precision')
      sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['recall'], ax=axes[i][0], label='val_recall')
      sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.scores[i]['validation']['f1'], ax=axes[i][0], label='val_f1')        

      axes[i][0].legend() 
      axes[i][0].set_title('Fold {} Validation Classification Report'.format(i), fontsize=14)

      # Loss curve
      sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[0].history.history['loss'], ax=axes[i][1], label='train_loss')
      sns.lineplot(x=np.arange(1, self.epochs + 1), y=clf.models[0].history.history['val_loss'], ax=axes[i][1], label='val_loss')

      axes[i][1].legend() 
      axes[i][1].set_title('Fold {} Train / Validation Loss'.format(i), fontsize=14)

      for j in range(2):
        axes[i][j].set_xlabel('Epoch', size=12)
        axes[i][j].tick_params(axis='x', labelsize=12)
        axes[i][j].tick_params(axis='y', labelsize=12)

        plt.show()


  def predict(self, X):
    X_test_encoded = self.encode(X['cleaned_text'].str.lower())
    y_pred = np.zeros((X_test_encoded[0].shape[0], 1))

    for model in self.models:
      y_pred += model.predict(X_test_encoded) / len(self.models)

    return y_pred

In [39]:
clf = DisasterDetector(bert_layer, max_seq_length=128, lr=0.0001, epochs=10, batch_size=32)
clf.train(train_df)

ValueError: ignored